# **PowerPoint**

## **Librerías**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.options.display.max_columns = False

In [3]:
import requests

In [5]:
from datetime import datetime, timedelta
import winsound

In [6]:
import json

## **Variables Generales**

In [7]:
datetime_now = datetime.now()
dia = int(str(datetime.now()).replace('-', '')[:8])

## **1. Llamada de los datos**

In [14]:
try:
    df = pd.read_csv('./Datos/currencies.csv')
except:
    full_list_url = 'https://finance.yahoo.com/currencies'
    full_list_page = requests.get(full_list_url)

    df = pd.read_html(full_list_page.text)[0].drop_duplicates()
    
    df.to_csv('./Datos/currencies.csv', index=False)

In [15]:
df.sample()

,Symbol,Name,Last Price,Change,% Change,52 Week Range,Day Chart,pct_change
2,GBPUSD=X,USD/GBP,1.2283,-0.0049,-0.40%,NaN,NaN,-0.4


## **2. Limpieza y exploración**

In [16]:
df['pct_change'] = df['% Change'].str.slice(stop=-1).astype('float')